---
title: Inline Interactivity in Jupyter using MyST
subtitle: Using inline widgets in JupyterLab markdown cells
author:
  - name: Rowan Cockett
    affiliations: Executable Books; Curvenote
    orcid: 0000-0002-7859-8394
    email: rowan@curvenote.com
  - name: Angus Hollands
    affiliations: Executable Books
  - name: Steve Purves
    affiliations: Executable Books; Curvenote
date: 2023/02/20
---

The [JupyterLab MyST extension](https://github.com/executablebooks/jupyterlab-myst) allows you to have MyST renderer in your markdown cells that includes interactivity and inline-evaluation.

This is done through the `` {eval}`1+1` `` role, which results in {eval}`1+1`. The extension can also access variables in the kernel.

:::{warning} Syntax is Subject to Change
:class: dropdown
The current syntax is based on a myst role, which may change in the future -- for example to be closer to an inline expression `${}`. This will go through a MyST enhancement proposal, and the `{eval}` role will likely still be supported.
:::

In [2]:
import numpy as np
array = np.arange(4)

{eval}`1+12`

Let's consider the following array: {eval}`array`.

We can compute the total: {eval}`array.sum()` and the maximum value is {eval}`array.max()`.

This is done through using MyST syntax: `` {eval}`array.sum()` `` and `` {eval}`array.max()` ``, respectively. 

Let's consider the following array: {eval}`x`

## Displaying text values computed by widgets

The example below builds a small widget that updates several values, and we then display them in markdown cells. We use a [`widgets.Label` object](https://ipywidgets.readthedocs.io/en/stable/examples/Widget%20List.html?#Label), which stores a _string_ value and displays it as formatted html.

In [3]:
import ipywidgets as widgets

cookiesSlider = widgets.IntSlider(min=0, max=30, step=1, value=10, description="Cookies: ")
cookiesText = widgets.BoundedIntText(
    value=10,
    min=0,
    max=30,
    step=1,
)
widgetLink = widgets.jslink((cookiesSlider, 'value'), (cookiesText, 'value'))

caloriesPerCookie = 50
dailyCalories = 2100

calories = widgets.Label(value=f'{cookiesSlider.value * caloriesPerCookie}')
def fc(n):
    calories.value = f'{n["owner"].value * caloriesPerCookie}'
cookiesSlider.observe(fc)

def pct_cookie(cookies):
    return f"{cookies * caloriesPerCookie / dailyCalories * 100:.1f}"

percent = widgets.Label(value=pct_cookie(cookiesSlider.value))

def fp(change):
    percent.value = pct_cookie(change['new'])
    
cookiesSlider.observe(fp, 'value')

:::{important} Calculate Consuming Cookie Calories 🍪 🧮
When you eat {eval}`cookiesText` cookies, you consume {eval}`calories` calories. 

That's {eval}`percent`% of your recommended daily calories.

:::

You can also edit this through a slider if you want: {eval}`cookiesSlider`

:::{note} Improvements
:class: dropdown
There is a lot to improve both for formatting (e.g. format the numbering) and for the verbosity of the above code for linking and calculating the widgets.
These are probably going to be a combination of a MyST library for working with simple inline widgets and showing numbers, as well as improvements and tweaks to the inline renderers for the mime bundles.

But you can already do things! 🚀
:::

The integral {eval}`I` can be analytically computed

{eval}`I` = {eval}`I.doit()`.

:::{warning} Math rendering is still unstable

We've noticed issues with this feature - rendering is slow and sometimes weird glitches orccur. [We're tracking it](https://github.com/executablebooks/jupyterlab-myst/issues/96).
:::